<a href="https://colab.research.google.com/github/adiel2012/pythorch-for-deeplearning/blob/main/notebooks/05_complete_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 5: Complete Examples and Projects

This notebook contains end-to-end implementations of common deep learning projects.

## Projects Covered
1. **Image Classification**: Complete CNN training pipeline
2. **Regression**: Neural network for continuous prediction
3. **Text Classification**: Simple NLP with embeddings
4. **Autoencoder**: Dimensionality reduction and reconstruction
5. **GAN**: Simple Generative Adversarial Network

## Setup and Installation

In [ ]:
# Install and import necessary libraries
try:
    import torch
    print(f"PyTorch version: {torch.__version__}")
except ImportError:
    !pip install torch torchvision torchaudio
    import torch
    print(f"PyTorch installed. Version: {torch.__version__}")

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_regression, make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

# Set device and random seed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
torch.manual_seed(42)
np.random.seed(42)

# Set matplotlib style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette("husl")

## Project 1: Complete Image Classification Pipeline

End-to-end image classification with data loading, model training, and evaluation.

In [ ]:
class ImageClassifier(nn.Module):
    """Complete image classification model"""
    
    def __init__(self, num_classes=10):
        super(ImageClassifier, self).__init__()
        
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.25),
            
            # Block 2
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.25),
            
            # Block 3
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((4, 4))
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Training function
def train_classifier(model, train_loader, val_loader, num_epochs=10):
    """Complete training function with validation"""
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)
    
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_val_acc = 0.0
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        # Calculate metrics
        train_acc = 100 * train_correct / train_total
        val_acc = 100 * val_correct / val_total
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        
        # Update history
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_acc)
        
        # Learning rate scheduling
        scheduler.step(avg_val_loss)
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_classifier.pth')
        
        print(f'Epoch [{epoch+1}/{num_epochs}] '
              f'Train Loss: {avg_train_loss:.4f} '
              f'Train Acc: {train_acc:.2f}% '
              f'Val Loss: {avg_val_loss:.4f} '
              f'Val Acc: {val_acc:.2f}%')
    
    return history

# Create synthetic dataset (replace with real data)
def create_synthetic_image_data(n_samples=1000, n_classes=5):
    """Create synthetic image data for demonstration"""
    
    # Generate random images
    images = torch.randn(n_samples, 3, 32, 32)
    
    # Add some patterns based on class
    labels = torch.randint(0, n_classes, (n_samples,))
    
    for i, label in enumerate(labels):
        # Add class-specific patterns
        if label == 0:  # Red channel dominant
            images[i, 0] += 0.5
        elif label == 1:  # Green channel dominant
            images[i, 1] += 0.5
        elif label == 2:  # Blue channel dominant
            images[i, 2] += 0.5
        elif label == 3:  # Checkerboard pattern
            for x in range(0, 32, 4):
                for y in range(0, 32, 4):
                    if (x//4 + y//4) % 2 == 0:
                        images[i, :, x:x+2, y:y+2] += 0.3
        else:  # Bright center
            images[i, :, 12:20, 12:20] += 0.4
    
    return images, labels

# Prepare data
print("=== Image Classification Project ===")
print("Creating synthetic dataset...")

images, labels = create_synthetic_image_data(2000, 5)
dataset = TensorDataset(images, labels)

# Split dataset
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Train model
model = ImageClassifier(num_classes=5).to(device)
print(f"\nModel parameters: {sum(p.numel() for p in model.parameters()):,}")

print("\nStarting training...")
history = train_classifier(model, train_loader, val_loader, num_epochs=5)

# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss plot
ax1.plot(history['train_loss'], label='Training Loss', linewidth=2)
ax1.plot(history['val_loss'], label='Validation Loss', linewidth=2)
ax1.set_title('Training and Validation Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Accuracy plot
ax2.plot(history['train_acc'], label='Training Accuracy', linewidth=2)
ax2.plot(history['val_acc'], label='Validation Accuracy', linewidth=2)
ax2.set_title('Training and Validation Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nBest validation accuracy: {max(history['val_acc']):.2f}%")

## Project 2: Neural Network Regression

Predicting continuous values with a feed-forward neural network.

In [ ]:
class RegressionNet(nn.Module):
    """Neural network for regression tasks"""
    
    def __init__(self, input_size, hidden_sizes, output_size=1, dropout_rate=0.2):
        super(RegressionNet, self).__init__()
        
        layers = []
        prev_size = input_size
        
        # Hidden layers
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        
        # Output layer (no activation for regression)
        layers.append(nn.Linear(prev_size, output_size))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def train_regressor(model, train_loader, val_loader, num_epochs=50):
    """Train regression model"""
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
    
    history = {'train_loss': [], 'val_loss': [], 'train_r2': [], 'val_r2': []}
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0.0
        train_preds, train_targets = [], []
        
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_preds.extend(outputs.cpu().detach().numpy())
            train_targets.extend(targets.cpu().detach().numpy())
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_preds, val_targets = [], []
        
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                
                val_loss += loss.item()
                val_preds.extend(outputs.cpu().numpy())
                val_targets.extend(targets.cpu().numpy())
        
        # Calculate R² scores
        train_r2 = r2_score(train_targets, train_preds)
        val_r2 = r2_score(val_targets, val_preds)
        
        # Update history
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['train_r2'].append(train_r2)
        history['val_r2'].append(val_r2)
        
        scheduler.step()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}] '
                  f'Train Loss: {avg_train_loss:.6f} '
                  f'Val Loss: {avg_val_loss:.6f} '
                  f'Train R²: {train_r2:.4f} '
                  f'Val R²: {val_r2:.4f}')
    
    return history

# Generate regression data
print("=== Neural Network Regression Project ===")
print("Generating regression dataset...")

# Create synthetic regression data
X, y = make_regression(n_samples=2000, n_features=20, n_informative=15, 
                      noise=0.1, random_state=42)

# Standardize features
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

# Train-validation split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42)

# Convert to tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1)
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.FloatTensor(y_val).unsqueeze(1)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).unsqueeze(1)

# Create data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"Test samples: {len(X_test)}")
print(f"Input features: {X_train.shape[1]}")

# Create and train model
model = RegressionNet(
    input_size=X_train.shape[1],
    hidden_sizes=[128, 64, 32],
    output_size=1,
    dropout_rate=0.2
).to(device)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print("\nStarting training...")

history = train_regressor(model, train_loader, val_loader, num_epochs=50)

# Test the model
model.eval()
test_preds = []
test_targets = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        test_preds.extend(outputs.cpu().numpy())
        test_targets.extend(targets.cpu().numpy())

# Calculate final metrics
test_r2 = r2_score(test_targets, test_preds)
test_mse = mean_squared_error(test_targets, test_preds)

print(f"\nFinal Test Results:")
print(f"R² Score: {test_r2:.4f}")
print(f"MSE: {test_mse:.6f}")
print(f"RMSE: {np.sqrt(test_mse):.6f}")

# Plot results
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Loss curves
ax1.plot(history['train_loss'], label='Training Loss')
ax1.plot(history['val_loss'], label='Validation Loss')
ax1.set_title('Loss Curves')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('MSE Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# R² curves
ax2.plot(history['train_r2'], label='Training R²')
ax2.plot(history['val_r2'], label='Validation R²')
ax2.set_title('R² Score Curves')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('R² Score')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Predictions vs actual
ax3.scatter(test_targets, test_preds, alpha=0.6)
ax3.plot([min(test_targets), max(test_targets)], 
         [min(test_targets), max(test_targets)], 'r--', linewidth=2)
ax3.set_title(f'Predictions vs Actual (R² = {test_r2:.4f})')
ax3.set_xlabel('Actual Values')
ax3.set_ylabel('Predicted Values')
ax3.grid(True, alpha=0.3)

# Residuals
residuals = np.array(test_targets).flatten() - np.array(test_preds).flatten()
ax4.scatter(test_preds, residuals, alpha=0.6)
ax4.axhline(y=0, color='r', linestyle='--', linewidth=2)
ax4.set_title('Residual Plot')
ax4.set_xlabel('Predicted Values')
ax4.set_ylabel('Residuals')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Project 3: Simple Autoencoder

Dimensionality reduction and data reconstruction using autoencoders.

In [ ]:
class Autoencoder(nn.Module):
    """Simple autoencoder for dimensionality reduction"""
    
    def __init__(self, input_size, encoding_size):
        super(Autoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_size, input_size // 2),
            nn.ReLU(),
            nn.Linear(input_size // 2, input_size // 4),
            nn.ReLU(),
            nn.Linear(input_size // 4, encoding_size),
            nn.ReLU()  # Ensure positive encoding
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(encoding_size, input_size // 4),
            nn.ReLU(),
            nn.Linear(input_size // 4, input_size // 2),
            nn.ReLU(),
            nn.Linear(input_size // 2, input_size),
            nn.Tanh()  # Output in [-1, 1]
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
    def encode(self, x):
        return self.encoder(x)

def train_autoencoder(model, dataloader, num_epochs=100):
    """Train autoencoder"""
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    
    losses = []
    
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        
        for data in dataloader:
            inputs = data[0].to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, inputs)  # Reconstruct input
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(dataloader)
        losses.append(avg_loss)
        scheduler.step()
        
        if (epoch + 1) % 20 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.6f}')
    
    return losses

# Create synthetic high-dimensional data
print("=== Autoencoder Project ===")
print("Generating high-dimensional data...")

# Generate data with some underlying structure
n_samples = 1000
n_features = 100
encoding_size = 10

# Create data with intrinsic lower dimensionality
np.random.seed(42)
latent_data = np.random.randn(n_samples, encoding_size)
mixing_matrix = np.random.randn(encoding_size, n_features)
high_dim_data = latent_data @ mixing_matrix
high_dim_data += 0.1 * np.random.randn(n_samples, n_features)  # Add noise

# Normalize data to [-1, 1]
scaler = StandardScaler()
high_dim_data = scaler.fit_transform(high_dim_data)
high_dim_data = np.tanh(high_dim_data)  # Squash to [-1, 1]

# Create dataset
data_tensor = torch.FloatTensor(high_dim_data)
dataset = TensorDataset(data_tensor)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

print(f"Data shape: {high_dim_data.shape}")
print(f"Encoding dimension: {encoding_size}")
print(f"Compression ratio: {n_features/encoding_size:.1f}:1")

# Create and train autoencoder
autoencoder = Autoencoder(n_features, encoding_size).to(device)
print(f"\nAutoencoder parameters: {sum(p.numel() for p in autoencoder.parameters()):,}")

print("\nStarting training...")
losses = train_autoencoder(autoencoder, dataloader, num_epochs=100)

# Test reconstruction
autoencoder.eval()
with torch.no_grad():
    sample_data = data_tensor[:100].to(device)
    reconstructed = autoencoder(sample_data)
    encoded_rep = autoencoder.encode(sample_data)
    
    reconstruction_error = F.mse_loss(reconstructed, sample_data).item()
    print(f"\nReconstruction MSE: {reconstruction_error:.6f}")

# Visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Training loss
ax1.plot(losses, linewidth=2)
ax1.set_title('Autoencoder Training Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('MSE Loss')
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# Original vs reconstructed (first sample)
sample_idx = 0
original = sample_data[sample_idx].cpu().numpy()
recon = reconstructed[sample_idx].cpu().numpy()

ax2.plot(original[:50], label='Original', linewidth=2, alpha=0.7)
ax2.plot(recon[:50], label='Reconstructed', linewidth=2, alpha=0.7)
ax2.set_title(f'Sample Reconstruction (first 50 features)')
ax2.set_xlabel('Feature Index')
ax2.set_ylabel('Value')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Encoding space (2D projection using PCA if needed)
encodings = encoded_rep.cpu().numpy()
if encoding_size >= 2:
    ax3.scatter(encodings[:, 0], encodings[:, 1], alpha=0.6, s=20)
    ax3.set_title('Encoding Space (first 2 dimensions)')
    ax3.set_xlabel('Encoding Dim 1')
    ax3.set_ylabel('Encoding Dim 2')
    ax3.grid(True, alpha=0.3)
else:
    ax3.hist(encodings[:, 0], bins=30, alpha=0.7)
    ax3.set_title('Encoding Distribution')
    ax3.set_xlabel('Encoding Value')
    ax3.set_ylabel('Frequency')
    ax3.grid(True, alpha=0.3)

# Reconstruction scatter plot
original_flat = sample_data.cpu().numpy().flatten()
reconstructed_flat = reconstructed.cpu().numpy().flatten()

ax4.scatter(original_flat, reconstructed_flat, alpha=0.3, s=1)
ax4.plot([original_flat.min(), original_flat.max()], 
         [original_flat.min(), original_flat.max()], 'r--', linewidth=2)
ax4.set_title('Original vs Reconstructed Values')
ax4.set_xlabel('Original Values')
ax4.set_ylabel('Reconstructed Values')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal reconstruction error: {reconstruction_error:.6f}")
print(f"Compression achieved: {n_features} → {encoding_size} dimensions")

## Project 4: Simple GAN

Basic Generative Adversarial Network for generating synthetic data.

In [ ]:
class Generator(nn.Module):
    """Generator network for GAN"""
    
    def __init__(self, noise_dim, output_dim):
        super(Generator, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(noise_dim, 128),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(128),
            
            nn.Linear(128, 256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),
            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(512),
            
            nn.Linear(512, output_dim),
            nn.Tanh()  # Output in [-1, 1]
        )
    
    def forward(self, x):
        return self.network(x)

class Discriminator(nn.Module):
    """Discriminator network for GAN"""
    
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(128, 1),
            nn.Sigmoid()  # Probability [0, 1]
        )
    
    def forward(self, x):
        return self.network(x)

def train_gan(generator, discriminator, dataloader, num_epochs=200):
    """Train GAN with alternating updates"""
    
    # Loss function
    criterion = nn.BCELoss()
    
    # Optimizers
    g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    
    # Training history
    history = {'d_loss': [], 'g_loss': [], 'd_acc': []}
    
    noise_dim = 100
    
    for epoch in range(num_epochs):
        epoch_d_loss = 0.0
        epoch_g_loss = 0.0
        epoch_d_acc = 0.0
        num_batches = 0
        
        for real_data in dataloader:
            batch_size = real_data[0].size(0)
            real_data = real_data[0].to(device)
            
            # Labels
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)
            
            # =====================
            # Train Discriminator
            # =====================
            d_optimizer.zero_grad()
            
            # Real data
            d_real_output = discriminator(real_data)
            d_real_loss = criterion(d_real_output, real_labels)
            
            # Fake data
            noise = torch.randn(batch_size, noise_dim).to(device)
            fake_data = generator(noise)
            d_fake_output = discriminator(fake_data.detach())
            d_fake_loss = criterion(d_fake_output, fake_labels)
            
            # Total discriminator loss
            d_loss = d_real_loss + d_fake_loss
            d_loss.backward()
            d_optimizer.step()
            
            # ==================
            # Train Generator
            # ==================
            g_optimizer.zero_grad()
            
            # Generate fake data and try to fool discriminator
            noise = torch.randn(batch_size, noise_dim).to(device)
            fake_data = generator(noise)
            d_fake_output = discriminator(fake_data)
            g_loss = criterion(d_fake_output, real_labels)  # Want discriminator to think it's real
            
            g_loss.backward()
            g_optimizer.step()
            
            # Statistics
            epoch_d_loss += d_loss.item()
            epoch_g_loss += g_loss.item()
            
            # Discriminator accuracy
            d_real_acc = (d_real_output > 0.5).float().mean().item()
            d_fake_acc = (d_fake_output <= 0.5).float().mean().item()
            epoch_d_acc += (d_real_acc + d_fake_acc) / 2
            
            num_batches += 1
        
        # Update history
        history['d_loss'].append(epoch_d_loss / num_batches)
        history['g_loss'].append(epoch_g_loss / num_batches)
        history['d_acc'].append(epoch_d_acc / num_batches)
        
        if (epoch + 1) % 50 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}] '
                  f'D Loss: {history["d_loss"][-1]:.4f} '
                  f'G Loss: {history["g_loss"][-1]:.4f} '
                  f'D Acc: {history["d_acc"][-1]:.4f}')
    
    return history

# Create target distribution (mixture of Gaussians)
print("=== Simple GAN Project ===")
print("Creating target distribution...")

def create_target_distribution(n_samples=2000, data_dim=2):
    """Create a mixture of Gaussians as target distribution"""
    
    # Three Gaussian clusters
    cluster1 = np.random.multivariate_normal([2, 2], [[0.5, 0], [0, 0.5]], n_samples // 3)
    cluster2 = np.random.multivariate_normal([-2, 2], [[0.5, 0], [0, 0.5]], n_samples // 3)
    cluster3 = np.random.multivariate_normal([0, -2], [[0.5, 0], [0, 0.5]], n_samples // 3)
    
    data = np.vstack([cluster1, cluster2, cluster3])
    
    # Add more dimensions if needed
    if data_dim > 2:
        extra_dims = np.random.randn(data.shape[0], data_dim - 2) * 0.1
        data = np.hstack([data, extra_dims])
    
    # Normalize to [-1, 1]
    data = np.tanh(data / 2)
    
    return data

# Generate target data
data_dim = 2  # Keep 2D for easy visualization
target_data = create_target_distribution(2000, data_dim)

print(f"Target data shape: {target_data.shape}")
print(f"Data range: [{target_data.min():.2f}, {target_data.max():.2f}]")

# Create dataset
target_tensor = torch.FloatTensor(target_data)
dataset = TensorDataset(target_tensor)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Create GAN
noise_dim = 100
generator = Generator(noise_dim, data_dim).to(device)
discriminator = Discriminator(data_dim).to(device)

print(f"\nGenerator parameters: {sum(p.numel() for p in generator.parameters()):,}")
print(f"Discriminator parameters: {sum(p.numel() for p in discriminator.parameters()):,}")

# Train GAN
print("\nStarting GAN training...")
history = train_gan(generator, discriminator, dataloader, num_epochs=200)

# Generate samples
generator.eval()
with torch.no_grad():
    noise = torch.randn(1000, noise_dim).to(device)
    generated_data = generator(noise).cpu().numpy()

print(f"\nGenerated data shape: {generated_data.shape}")
print(f"Generated data range: [{generated_data.min():.2f}, {generated_data.max():.2f}]")

# Visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Training curves
ax1.plot(history['d_loss'], label='Discriminator Loss', linewidth=2)
ax1.plot(history['g_loss'], label='Generator Loss', linewidth=2)
ax1.set_title('GAN Training Losses')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Discriminator accuracy
ax2.plot(history['d_acc'], linewidth=2, color='green')
ax2.axhline(y=0.5, color='red', linestyle='--', linewidth=2, label='Random Guess')
ax2.set_title('Discriminator Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Data distribution comparison (2D)
if data_dim == 2:
    ax3.scatter(target_data[:, 0], target_data[:, 1], alpha=0.6, s=20, label='Real Data')
    ax3.set_title('Real Data Distribution')
    ax3.set_xlabel('Dimension 1')
    ax3.set_ylabel('Dimension 2')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    ax4.scatter(generated_data[:, 0], generated_data[:, 1], alpha=0.6, s=20, 
               color='orange', label='Generated Data')
    ax4.set_title('Generated Data Distribution')
    ax4.set_xlabel('Dimension 1')
    ax4.set_ylabel('Dimension 2')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
else:
    # For higher dimensions, show marginal distributions
    ax3.hist(target_data[:, 0], bins=30, alpha=0.7, label='Real Data')
    ax3.hist(generated_data[:, 0], bins=30, alpha=0.7, label='Generated Data')
    ax3.set_title('Marginal Distribution (Dim 1)')
    ax3.set_xlabel('Value')
    ax3.set_ylabel('Frequency')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    ax4.hist(target_data[:, 1], bins=30, alpha=0.7, label='Real Data')
    ax4.hist(generated_data[:, 1], bins=30, alpha=0.7, label='Generated Data')
    ax4.set_title('Marginal Distribution (Dim 2)')
    ax4.set_xlabel('Value')
    ax4.set_ylabel('Frequency')
    ax4.legend()
    ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal discriminator accuracy: {history['d_acc'][-1]:.4f}")
print("Note: Ideal discriminator accuracy should be around 0.5 (can't distinguish real from fake)")

## Summary: Complete Deep Learning Projects

In this comprehensive notebook, we implemented five complete deep learning projects:

### 1. Image Classification Pipeline
- **Architecture**: Multi-block CNN with batch normalization and dropout
- **Features**: Data augmentation, validation monitoring, learning rate scheduling
- **Key Concepts**: Transfer learning potential, regularization techniques

### 2. Neural Network Regression
- **Architecture**: Deep feed-forward network with batch normalization
- **Evaluation**: R² score, MSE, residual analysis
- **Applications**: Continuous value prediction, function approximation

### 3. Autoencoder for Dimensionality Reduction
- **Architecture**: Symmetric encoder-decoder with bottleneck
- **Applications**: Data compression, denoising, anomaly detection
- **Insights**: Learned representations, reconstruction quality

### 4. Simple Generative Adversarial Network
- **Architecture**: Generator and discriminator in adversarial training
- **Concepts**: Minimax game, Nash equilibrium, mode collapse
- **Applications**: Data generation, distribution matching

### Key Implementation Patterns

1. **Model Design**:
   - Modular architecture with clear forward pass
   - Appropriate activation functions and normalization
   - Regularization techniques (dropout, weight decay)

2. **Training Loop**:
   - Proper gradient management (zero_grad, backward, step)
   - Validation monitoring and early stopping
   - Learning rate scheduling

3. **Data Handling**:
   - Efficient data loading with DataLoader
   - Appropriate train/validation/test splits
   - Data preprocessing and normalization

4. **Evaluation and Visualization**:
   - Comprehensive metrics for different tasks
   - Training curve monitoring
   - Result visualization and interpretation

### Best Practices Demonstrated

- **Device Management**: Proper GPU utilization
- **Reproducibility**: Random seed setting
- **Model Saving**: Checkpoint best models
- **Monitoring**: Track multiple metrics
- **Regularization**: Prevent overfitting
- **Architecture Design**: Match model to task

### Next Steps for Real Projects

1. **Use Real Datasets**: Replace synthetic data with domain-specific datasets
2. **Hyperparameter Tuning**: Implement grid search or Bayesian optimization
3. **Advanced Architectures**: Explore ResNet, Transformers, etc.
4. **Transfer Learning**: Use pre-trained models
5. **Production Deployment**: Model serving and monitoring
6. **Advanced Training**: Mixed precision, distributed training

These projects provide a solid foundation for building more complex deep learning applications!